### 第二十章 网络通讯

#### 本章内容

1. 使用Python套接字完成一个简单的一对一聊天窗口
2. 使用多线程技术优化交互流程
3. 调用腾讯云API实现基于人工智能的机器翻译

#### 1. 网络基础回顾

在第十章中，我们详细探讨了计算机网络的基本知识。为了更好地理解本章的实践内容，我们将对这些知识进行快速回顾。

随着信息技术的飞速发展，计算机在全球范围内相互交流、传递数据显得尤为重要。计算机通过网络实现数据的互传，使得不同的设备之间能够进行信息共享。这种数据交换的基础就是计算机网络，它将各类设备连接起来形成一个完整的系统。

计算机网络的连接方式主要有以下几种：

1. **局域网 (LAN)**：用于在较小的范围内（如家庭、学校或公司）连接多台计算机。局域网内的设备通常通过以太网交换机连接。

2. **广域网 (WAN)**：覆盖地理范围较广，通常连接不同城市或国家的局域网，互联网本质上就是一个大型的广域网。

3. **个人局域网 (PAN)**：用于连接个人设备，如智能手机、平板和笔记本电脑，通常通过蓝牙或Wi-Fi技术实现。

4. **点对点网络 (P2P)**：计算机直接连接，不需要中央服务器。每台计算机都能充当客户端与服务器进行直接通信。

在计算机网络中，每个设备都有一个唯一的身份标识：

- **MAC地址**：每一台连接到网络的设备都有一个唯一的物理地址，称为MAC地址（媒体访问控制地址）。MAC地址是由厂商在网络接口卡生产时刻生成的，它是一个48位的二进制数字，通常以十六进制形式表示，例如：`00:1A:2B:3C:4D:5E`，用于在局域网中识别设备。

- **IP地址**：与MAC地址不同，IP地址是用于在互联网中识别设备的逻辑地址。IP地址分为两种类型IPv4与IPv6。IPv4地址通常以四个十进制数字表示（每个数字范围在0到255之间），例如：`192.168.1.1`。IPv6地址则使用十六进制表示，能够提供更多可用地址。

- **端口**：在计算机网络中（同一台计算机上），端口是用于区分不同服务和应用程序的逻辑接口。每个运行在计算机上的服务都通过一个特定的端口进行通信。常见的端口包括：
    - **HTTP（80）**：用于访问网页。
    - **HTTPS（443）**：用于安全的网页访问。
    - **FTP（21）**：用于文件传输。
    - **SMTP（25）**：用于发送电子邮件。
    - **SSH（22）**：用于安全远程登录。

端口号的使用，使得同一台计算机可以同时运行多个网络应用程序，确保它们之间的数据传输不相互干扰。

在MAC地址、IP地址与端口三个底层技术之上，我们创造了更多的应用数据通讯协议，通过这些数据协议进而创造各种各样的网络应用。

TCP（传输控制协议）和UDP（用户数据报协议）是两个至关重要的协议，用于基本的数据传输。

- **TCP协议**：
   - 是一种面向连接的协议，提供可靠的数据传输。
   - 在发送数据前，先建立连接，确保数据包的顺序和完整性、丢包重传等机制。
   - **应用场景**：适用于需要数据完整性和顺序传递的场景，如网页浏览和文件下载。

- **UDP协议**：
   - 是一种无连接的协议，数据传输速率较快，但不保证数据的顺序和完整性。
   - 不进行连接建立，数据包一旦发送即丢失将不再重传。
   - **应用场景**：适合用于实时应用，如视频会议、在线游戏等，因为这些场景通常更关注速度而非可靠性。

#### 2. 套接字技术与Python实现

在网络通讯中，套接字是实现计算机间数据传输的基础工具。在前面的部分中，我们已经了解了网络通讯的基本知识，而套接字正是使这种通讯成为可能的核心技术。无论是开发网页、聊天软件还是任何需要网络交互的应用程序，套接字都扮演着至关重要的角色。

**套接字**（Socket）是一种软件结构，用于实现不同计算机之间的网络通讯。它提供了一个通用的接口，开发者可以通过它发送和接收数据。套接字可以看作是一个网络接口，允许程序通过网络发送和接收数据流。每个套接字都有一个特定的地址，包括其IP地址和端口号，这是与其他设备进行数据交互的基础。

套接字的基本使用流程如下（伪代码）：

```
创建套接字：确定是TCP还是UDP
   socket = create_socket(protocol, type)
绑定套接字到地址：将IP与端口号绑定到套接字身上
   bind(socket, address)
创建连接（对于客户端）
   connect(socket, address)
监听连接（对于服务器）
   listen(socket)
接受连接（对于服务器）
   client_socket = accept(socket)
发送和接收数据
   send(client_socket, data)
   receive(client_socket)
关闭套接字
   close(socket)
```

需要注意的是，套接字的概念是语言无关的，意味着几乎所有的编程语言都可以使用套接字进行网络通讯。

在网络编程中，TCP（传输控制协议）和UDP（用户数据报协议）是两种不同的传输层协议，各自有其适用情况和特性。

| 特点                  | TCP                                     | UDP                                  |  
|---------------------|-----------------------------------------|-----------------------------------------|  
| **连接方式（服务器）**| 需要建立连接: `tcp_server_socket.accept()`    | 无需建立连接: 直接发送数据 |  
| **发送数据**| `send(XXX)` | `sendto(XXX, addr)` |  
| **接收数据**| `recv()` | `recvfrom(addr)` |  
| **关闭套接字**| `close()` | `close()` |  
| 数据传输保证 | 确保数据顺序与可靠性 | 不保证顺序与可靠性 |
| 流控与拥塞控制 | 有流控与拥塞控制 | 无流控与拥塞控制 |
| 使用场景 | 文件传输、网页浏览 | 实时语音、视频会议、在线游戏 |
| 速度 | 较慢，因需要建立连接与确认数据到达   | 较快，无需建立连接 |

下面是使用Python实现UDP信息发送的例子，首先需要导入`socket`模块。

```python
import socket
# 创建UDP套接字
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# 定义目标地址和端口
target_address = ('localhost', 12345)
# 发送消息
message = "Hello, UDP!"
udp_socket.sendto(message.encode(), target_address)
# 关闭套接字
udp_socket.close()
```

在实际使用套接字时，必须是服务器端和客户端同时实现才可以。code/xxx中就是一个完整的UDP服务器和客户端的例子。

*注：实际上对于UDP而言，没有明确的服务器与客户端的区别，那方先启动那方就是服务器*

Jupyter无法同时运行两段代码，所以需要你自行将代码拷贝到两个py文件中运行。

```python
# UDP 服务器
import socket

# 创建UDP套接字
# AF_INET表示使用IPv4协议，SOCK_DGRAM表示使用UDP协议
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# 定义目标地址和端口
target_address = ('localhost', 12345)

# 发送消息
message = "Hello, UDP!"
udp_socket.sendto(message.encode(), target_address)

# 关闭套接字
udp_socket.close()
```

再接收端的代码示例可以是：

```python
# UDP 客户端
import socket

# 创建UDP套接字
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# 绑定地址和端口
local_address = ('localhost', 12345)
udp_socket.bind(local_address)

print("服务器已启动，等待消息...")

# 接收消息
data, addr = udp_socket.recvfrom(1024)
print(f"接收来自 {addr} 的消息: {data.decode()}")

# 关闭套接字
udp_socket.close()
```

#### 3. 多线程

在进行刚才的UDP服务器与客户端实验时，您有没有发现每次只能处理一个客户端的请求？每当一个客户端连接后，服务器必须等待处理完毕才能接收下一个请求。这就是单线程的工作机制。单线程操作还有许多，如文件读取、网络请求等，但在面对多个任务同时进行时，单线程无法满足性能和响应的需要。为了解决这些问题，我们必须使用多线程。

在计算机中，**进程**是运行中的程序的实例，操作系统通过调度进程来实现多个程序的并发执行。每个进程都有自己的内存空间和资源。相对而言，**线程**是进程中的执行单元，属于同一进程的多个线程可以共享该进程的内存和资源。

- **进程**：独立的工作单位，包含执行代码及其相关的数据，具有自己的内存空间。比如运行一个Word进程。
- **线程**：轻量级的工作单位，属于某个进程，能够共享该进程的资源和数据。比如Word中的拼写检查线程。

以下是多线程编程的基本框架伪代码：

```
定义线程任务函数
   def thread_task():
       # 任务逻辑
       ...
创建线程
   thread = create_thread(thread_task)
启动线程
   thread.start()
等待线程完成
   thread.join()
```

在多线程编程中，经常会遇到共享数据的问题。多个线程在访问共享数据时，容易产生数据竞争，导致意想不到的结果。为了避免这种情况，我们需要使用**互斥量（Mutex）**。

**互斥量**是一个同步原语，用于控制对共享资源的访问。当一个线程获得互斥量时，其他线程必须等待，直到该线程释放互斥量，从而确保在同一时刻只有一个线程访问共享资源。

想象一下在一家餐厅，厨师在厨房里，而服务员需要从厨房取食物。如果只有一个服务员（即单线程），他必须等菜做好才能取走，服务效率将会低下。如果有多个服务员（即多线程），他们可以同时从厨房取食物，极大提高了服务效率。

在这个过程中，如果所有服务员都试图同时进厨房取同一盘菜，就会造成混乱。为了解决这个问题，餐厅可以为厨房设置一个门（互斥量），每次只有一个服务员可以进入厨房取餐。

在Python中，我们可以使用`threading`模块来实现多线程。这是一个简单的多线程代码示例，展示了如何创建和启动线程。

In [ ]:
import threading
import time

# 定义线程任务
def thread_task(name):
    print(f"{name} 开始工作")
    time.sleep(2)  # 模拟任务耗时
    print(f"{name} 完成工作")

# 创建线程
thread1 = threading.Thread(target=thread_task, args=("线程1",))
thread2 = threading.Thread(target=thread_task, args=("线程2",))

# 启动线程
thread1.start()
thread2.start()

# 等待线程完成：确保主线程等待所有子线程完成后再输出“所有线程完成”。
thread1.join()
thread2.join()

print("所有线程完成")

除了直接使用Thread类型，我们还可以通过继承Thread并重载run函数的方式，创建一个自定义线程，下面的代码演示使用具有互斥锁的自定义线程。
下面的代码中创建了两个功能完全一样的线程类MyThread1与MyThread2，他们的区别主要在于使用lock的方式。

In [ ]:
import threading
import time

# 创建锁
lock = threading.Lock()
shared_data = 0  # 共享数据

# 定义线程类，继承自 threading.Thread
class MyThread1(threading.Thread):
    def __init__(self, name):
        super().__init__()  # 调用父类的构造函数
        self.name = name

    def run(self):
        global shared_data, lock
        for _ in range(10):
            with lock:
                shared_data += 1
                print(f"{self.name} 更新共享数据为: {shared_data}")
                time.sleep(2)

class MyThread2(threading.Thread):
    def __init__(self, name):
        super().__init__()  # 调用父类的构造函数
        self.name = name
        
    def run(self):
        global shared_data, lock
        for _ in range(10):
            lock.acquire()  # 加锁
            shared_data += 1
            print(f"{self.name} 更新共享数据为: {shared_data}")
            time.sleep(2)
            lock.release()  # 解锁

# 创建线程实例
thread1 = MyThread1("线程-1")
thread2 = MyThread2("线程-2")

# 获取线程信息
print(f"线程名称: {thread1.name}")
print(f"线程ID: {thread1.ident}")
print(f"线程是否在运行: {thread1.is_alive()}")

# 启动线程
thread1.start()
thread2.start()

# 等待所有线程完成
thread1.join()
thread2.join()

print("所有线程执行完毕")

#### 4. 聊天软件实现

在现代应用程序中，聊天软件是一个常见的需求，可以让用户之间进行实时交流。在本节中，我们将实现一个一对一的聊天软件，通过TCP协议进行网络通讯。我们将首先设计客户端和服务器端的核心代码段，并分析单线程的局限性，最后过渡到多线程的实现。

客户端的主要功能是连接到服务器，发送消息，并接收服务器的回信。

```
创建TCP套接字
连接到服务器
循环:
   用户输入消息
   发送消息到服务器
   接收来自服务器的回应
   如果用户输入"退出"，退出循环
关闭套接字
```

服务器端则负责监听客户端的连接请求，并处理接收到的消息。以下是服务器的核心功能伪代码：

```
创建TCP套接字
绑定IP地址和端口
启动监听
循环:
   接受客户端的连接
   循环接收消息:
      接收来自客户端的消息
      如果消息是"退出"，关闭连接
      打印消息并发送回信
关闭套接字
```

目前，我们的聊天软件设计使用的是单线程实现。正如你所看到的，这种方式在实现简单聊天功能时是可行的，但它有一些明显的局限性：

- **同时连接用户的限制**：目前服务器只能处理一个客户端的请求。当第一个客户端连接并发送消息时，服务器必须等待其完成交互，才能接收其他客户端的请求。
- **响应延迟**：服务器在处理一个客户端时无法响应其他客户端的消息，这会导致用户的体验不佳，特别是在用户较多时。

因此，为了提升性能和用户体验，我们需要将服务器改为多线程实现，以支持多个客户端同时进行消息交互。

在新的设计中，服务器端将为每个连接的客户端创建一个新的线程，确保能够并行处理多个客户端的请求。

```
定义每个客户端处理函数
    循环:
    接受客户端的连接
    循环接收消息:
        接收来自客户端的消息
        如果消息是"退出"，关闭连接
        打印消息并发送回信

创建TCP套接字
绑定IP地址和端口
启动监听
循环:
   为每个连接创建新线程，执行处理函数
关闭套接字
```

code/xxx中就是一个完整的聊天软件例子。运行时，需要先运行服务器端代码，然后再运行客户端代码。

Jupyter无法同时运行两段代码，所以需要你自行将代码拷贝到两个py文件中运行。

```python
# TCP 客户端
import socket

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 12345))

while True:
    message = input("请输入消息 (输入'退出'停止聊天): ")
    client_socket.send(message.encode())
    
    if message.lower() == '退出':
        break
    
    response = client_socket.recv(1024).decode()
    print(f"服务器返回的消息: {response}")

client_socket.close()
print("连接已关闭。")
```

这一段代码是TCP服务器端代码，并且使用了多线程技术。

```python
import socket
import threading

# 定义处理客户端连接的函数
def handle_client(client_socket, client_address):
    print(f"{client_address} 连接到服务器。")
    
    while True:
        try:
            # 接收消息
            message = client_socket.recv(1024).decode()
            if not message:  # 如果没有消息，表示连接已关闭
                print(f"{client_address} 断开连接。")
                break
            
            if message.lower() == '退出':  # 如果收到“退出”消息
                print(f"{client_address} 请求关闭连接。")
                break
            
            print(f"接收到来自 {client_address} 的消息: {message}")
            # 发送回信
            client_socket.send(f"服务器确认: {message}".encode())
        except Exception as e:
            print(f"处理 {client_address} 时发生错误: {e}")
            break

    client_socket.close()  # 关闭与客户端的连接

# 创建TCP套接字
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 12345))  # 绑定IP地址和端口
server_socket.listen(5)  # 启动监听，最多5个等待连接
print("服务器已经启动，等待连接...")

# 主循环，接受客户端连接并为每个连接创建新线程
while True:
    try:
        client_socket, client_address = server_socket.accept()  # 接受连接
        # 创建一个新线程处理该客户端
        client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
        client_thread.start()  # 启动线程
    except KeyboardInterrupt: # 使用Ctrl+C可以安全关闭服务器
        print("\n服务器正在关闭...")
        break

# 清理关闭
server_socket.close()
print("服务器已关闭。")
```

#### 5. HTTP协议

好TCP和UDP虽然提供了基本的网络传输能力，但在实际的互联网使用中，许多应用程序需要更高层次的协议来规范数据的传输与处理。如果只使用TCP或UDP传输数据，就会面临如何解析和处理数据的问题，这在不同的应用之间可能导致不必要的复杂性。因此，人们在TCP/UDP协议之上，定义了许多应用协议，其中HTTP（超文本传输协议）就是其中最为重要的一种。

**HTTP（HyperText Transfer Protocol）**是一个无状态的应用层协议，用于在客户端（如Web浏览器）和服务器之间传输超文本数据。HTTP可以支持多种数据格式，如文本、图像、视频等，因此它是Web世界中最常用的协议之一。

HTTP通讯的基本流程通常包括以下几个步骤：
- **客户端发送请求**：客户端（如浏览器）通过HTTP请求向服务器发送数据。
- **服务器处理请求**：服务器接收请求并进行处理。
- **服务器返回响应**：服务器将处理结果返回给客户端，该结果通常包含状态码和数据。
- **客户端接收响应**：客户端接收服务器的响应并进行相应处理。

在这其中，根据请求方式的不同，HTTP请求通常分为以下几种：
- **GET**：请求指定的资源，通常用于数据的读取。
- **POST**：向服务器提交数据，通常用于创建或更新资源。
- **PUT**：更新指定的资源，通常用于替换数据。
- **DELETE**：删除指定的资源。
- **HEAD**：请求资源的元信息，类似于GET，但不返回实际内容。
- **OPTIONS**：用于查询服务器支持的HTTP方法。

使用Python的`requests`库，可以实现简单的HTTP通讯，例如下面的代码示例使用GET请求请求 `https://cn.bing.com/` 的内容：

In [ ]:
import requests

response = requests.get('https://cn.bing.com/')
print(response.text)  # 打印获取到的网页内容

随着互联网的发展，API（应用程序编程接口）的功能也从本地网络逐渐扩展到了网络世界。于是，人们基于HTTP协议设计了许多以HTTP为基础的网络API，以支持各种不同的服务，比如RESTful API和GraphQL API等，极大地丰富了互联网应用程序之间的互联互通。

#### 6. RESTful API

在互联网发展的初期，客户端和服务器之间的交互方式相对简单，但随着应用的不断复杂化，开发者面临诸多挑战。例如，如何设计一种简洁、可扩展且易于理解的接口，以便不同的程序能够顺利地进行数据交互？这些问题推动了网络API设计的发展。

在2000年前后，计算机科学家罗伊·菲尔丁（Roy Fielding）在其博士论文中提出了“表现层状态转移”（Representational State Transfer，简称REST）的概念，旨在解决传统API设计中的一些不足之处。RESTful API因此应运而生，成为了一种遵循REST原则的网络服务架构风格。

RESTful API的设计遵循以下几个基本原则：  
- **无状态**：每个请求都应该包含处理该请求所需的所有信息，服务器不应存储客户端的状态。这样可以简化服务器的结构和提高可伸缩性。
- **资源标识**：REST强调资源的概念，每个资源通过URL唯一标识。这使得每个资源都能通过特定的URI进行访问。
- **使用标准HTTP方法**：RESTful API使用HTTP协议，典型的请求方法包括：  
    - **GET**：获取资源
    - **POST**：创建新资源
    - **PUT**：更新现有资源
    - **DELETE**：删除资源
- **自描述消息**：每个请求和响应都包含足够的信息，以便接收方能够理解如何处理此请求数据或响应。不仅要使用标准的状态码，还需合理使用HTTP头部信息。
- **超媒体作为应用状态的引擎**：客户端应通过超链接（hyperlink）导航，服务器返回的响应应提供相关的链接，以引导客户端进行后续操作。

我们以一个用户管理系统为例，来对比传统API设计与RESTful API设计的区别。
在传统API设计中，通常使用动词来描述操作，而在RESTful API设计中，使用名词来描述资源，并通过HTTP方法来定义操作。以下是一个对比表格：

| 设计方式          | URL                             | 请求方法 | 是否明确资源 |  
|------------------|----------------------------------|----------|--------------|  
| **传统设计**     | `POST /api/createUser`          | POST     | 否           |  
|                  | `GET /api/getUser?id=1`         | GET      | 否           |  
|                  | `POST /api/updateUser`          | POST     | 否           |  
|                  | `POST /api/deleteUser`          | POST     | 否           |  
| **RESTful设计**  | `POST /users`                    | POST     | 是           |  
|                  | `GET /users/1`                   | GET      | 是           |  
|                  | `PUT /users/1`                   | PUT      | 是           |  
|                  | `DELETE /users/1`                | DELETE   | 是           |  

在现代基于云技术（网络技术）的开发过程中，HTTP请求与RESRful API已经成为了主流的网络通讯方式。
下面的例子，我们将通过调用腾讯云API实现基于人工智能的机器翻译。腾讯云提供了丰富的API接口，可以用于各种应用场景，包括自然语言处理、图像识别等。
在运行下面的例子前，你需要先在腾讯云官网注册账号，并获取API密钥。
API秘钥（或者叫API-KEY）是访问腾讯云API的凭证，通常包括一个`SecretId`和一个`SecretKey`。你可以在腾讯云控制台中找到这些信息。

```python
# 下面完整代码中，修改这一行的内容，并且填写你的ID与KEY
cred = credential.Credential("SecretId", "SecretKey", "Token")
```

在使用腾讯云API时，可以使用Python的`requests`库来发送HTTP请求。也可以使用腾讯进行封装后的`qcloudapi-sdk-python`库来简化API调用。
在使用前需要执行`pip install tencentcloud-sdk-python-tmt`命令安装腾讯云的SDK。


In [ ]:
import json
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.tmt.v20180321 import tmt_client, models
try:
    # 实例化一个认证对象，入参需要传入腾讯云账户 SecretId 和 SecretKey，此处还需注意密钥对的保密
    # 代码泄露可能会导致 SecretId 和 SecretKey 泄露，并威胁账号下所有资源的安全性
    # 以下代码示例仅供参考，建议采用更安全的方式来使用密钥
    # 请参见：https://cloud.tencent.com/document/product/1278/85305
    # 密钥可前往官网控制台 https://console.cloud.tencent.com/cam/capi 进行获取
    cred = credential.Credential("SecretId", "SecretKey", "Token")
    # 实例化一个http选项，可选的，没有特殊需求可以跳过
    httpProfile = HttpProfile()
    httpProfile.endpoint = "tmt.tencentcloudapi.com"

    # 实例化一个client选项，可选的，没有特殊需求可以跳过
    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    # 实例化要请求产品的client对象,clientProfile是可选的
    client = tmt_client.TmtClient(cred, "ap-beijing", clientProfile)

    # 实例化一个请求对象,每个接口都会对应一个request对象
    req = models.TextTranslateRequest()
    params = {
        "SourceText": "你好呀，庞兴庆",
        "Source": "zh",
        "Target": "en",
        "ProjectId": 0
    }
    req.from_json_string(json.dumps(params))

    # 返回的resp是一个TextTranslateResponse的实例，与请求对象对应
    resp = client.TextTranslate(req)
    # 输出json格式的字符串回包
    print(resp.to_json_string())

except TencentCloudSDKException as err:
    print(err)

#### 本章总结

##### 本章知识点汇总

1. **局域网 (LAN)**：用于在限定区域内（如家庭或学校）连接多台计算机，以实现数据共享。

2. **广域网 (WAN)**：连接较大地理范围的局域网，如城市或国家之间的网络，互联网就是一个广域网的例子。

3. **个人局域网 (PAN)**：专用于连接个人设备（如智能手机、平板、笔记本）的网络，通常通过蓝牙或Wi-Fi进行连接。

4. **MAC地址**：每台设备的唯一物理地址，用于在网络中标识设备，防止冲突。

5. **IP地址**：逻辑地址，用于在互联网中识别设备，有IPv4和IPv6两种格式。

6. **端口**：用于区分同一台计算机上不同服务的逻辑接口，常见的有HTTP（80），HTTPS（443），FTP（21）等。

7. **传输控制协议 (TCP)**：面向连接的协议，提供可靠的数据传输，保证数据顺序和完整性。

8. **用户数据报协议 (UDP)**：无连接协议，数据传输效率高，但不保证数据的顺序和完整性。

9. **超文本传输协议 (HTTP)**：无状态的应用层协议，主要用于在客户端和服务器之间传输超文本数据。

10. **套接字 (Socket)**：用于在不同计算机间进行数据传输的基础工具，提供发送和接收数据的接口。

11. **点对点网络 (P2P)**：计算机之间直接连接，无需中央服务器，允许每台计算机充当客户端和服务器。

12. **多线程**：允许程序同时处理多个任务的技术，常用于提高计算机程序的性能。

13. **RESTful API**：基于HTTP的网络应用程序接口，设计简洁易用，支持不同服务之间的数据交换。

##### 课后练习

1. **局域网 (LAN) 的主要特点是什么？**
   
2. **无连接的协议有哪些，UDP与TCP之间有什么区别？请举例说明。**

3. **IP地址的主要功能是什么？为什么在网络通讯中必须使用它？**

4. **解释 MAC 地址，并说明它在网络通信中的作用。**

5. **描述套接字的概念，为什么它是网络编程中的核心组件？**

6. **HTTP协议与HTTPS协议有什么区别？为什么在现代网站中通常使用HTTPS？**

7. **什么是多线程，为什么在网络编程中需要使用多线程？**

8. **设计一个简单的聊天应用程序架构，描述服务器端与客户端的工作流程。**

9. **什么是 RESTful API？请列出其主要特征并举例说明其应用。**

10. **解释点对点网络 (P2P) 的概念及其优缺点。**

11. **请描述在 Python 中如何使用请求库发送一个简单的 GET 请求，并处理返回结果。**


##### 扩展知识

##### 练习题提示

1. 局域网是用于在有限的地理范围内（如家庭或办公楼内）连接多台计算机的网络，通常传输速度高、延迟低。

2. 无连接协议如UDP不需要在发送数据前建立连接，适合实时应用，如视频流。TCP则是面向连接的协议，确保数据的可靠传输，适合需要完整性保证的应用，如文件传输。

3. IP地址用于唯一识别网络上的每台设备，确保数据能够准确地送达目标设备。

4. MAC地址是网络接口的物理地址，用于在局域网内识别设备，确保数据包能够准确到达相应设备。

5. 套接字是进行网络通讯的接口，允许程序通过网络发送和接收数据。它是网络编程的核心，可以理解为通信端点。

6. HTTPS是带有SSL/TLS加密的HTTP协议，提供数据传输的安全性，防止数据被窃取，因此现代网站更倾向于使用HTTPS以保护用户隐私和信息安全。

7. 多线程是指在同一进程中同时运行多个执行单元，可以提高程序的响应能力，并发处理多个网络请求，提高服务器效率。

8. 服务器负责监听客户端请求，接收消息并转发给目标客户端。客户端通过套接字连接服务器，发送消息并接收来自其他客户端的消息。

9. ESTful API是一种设计网络服务的架构风格，主要特征包括无状态、资源导向和使用标准HTTP方法。常见应用包括Web服务如Twitter API、GitHub API等。

10. P2P网络是一种去中心化的网络，每台计算机既可以作为客户端又可以作为服务器。优点是资源共享与故障容错性强，缺点是安全性和管理成本高。

11. 使用`requests`库发送GET请求的方法 `response = requests.get('https://api.example.com/data')`，要处理返回结果，可以使用 `data = response.json()` 来解析返回的JSON数据。

12. **远程操作端代码**:
```python
# 远程操作端代码
import socket
import time
import random

def main():
    server_address = ('localhost', 12345)  # 服务器地址
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(server_address)
    s.listen(1)  # 监听连接
    print(f"服务器启动，等待连接...")
    s, addr = s.accept()  # 接受连接
    print(f"连接来自: {addr}")
    
    while True:
        temperature = random.uniform(20, 35)  # 随机生成温度
        print(f"发送: {temperature:.2f} °C")
        s.sendall(f"{temperature:.2f}".encode('utf-8'))  # 发送数据
        time.sleep(2)  # 每隔2秒发送一次

if __name__ == "__main__":
    main()
```

**客户端代码**:
```python
# 客户端代码
# 客户端代码
import socket
import py5

latest_temperature = "N/A"

def setup():
    global sock
    py5.size(400, 200)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect(('localhost', 12345))
    py5.no_loop()

def draw():
    py5.background(255)
    
    # 显示当前温度
    py5.fill(0)
    py5.text_size(32)
    py5.text(f"temperature: {latest_temperature} °C", 50, 100)

def receive_temperature():
    global latest_temperature
    try:
        data = sock.recv(1024)
        if data:
            latest_temperature = data.decode('utf-8')
            print(f"接收: {latest_temperature} °C")
            py5.redraw()
    except Exception as e:
        print(f"错误: {e}")

def key_pressed():
    receive_temperature()  # 手动刷新温度数据

py5.run_sketch()
```